# 今さら聞けないGANシリーズ(3)
https://qiita.com/triwave33/items/a5b3007d31d28bc445c2

潜在変数と生成画像について，つまり多分大体モード崩壊について

## 生成モデル
生成モデル$P_{gen}$は各ピクセル値が任意の値をとる同時確率分布を求めること．  
$$ P_{gen} = p(x_1, x_2, x_3, ... x_{784}) $$
確率密度分布が分かれば，それに基づいてデータを人工的に生成できる．  

例えばサイコロなら，
$$ P(x) = 1/6 \ where \ (x = {1,2,3,4,5,6}) $$
という確率密度分布になっている．  
このような生成モデルが決定されたら，確率密度分布から$x$の値をサンプリングすることができる．確率分布に従ってサイコロを振ることに等しい．  

## GANの生成モデル
この$P(x)$をニューラルネットワークで近似してやろうというのがGANのGenerator  
潜在変数$z$が与えられた時に，$z$の条件付き確率分布$G(x|z)$(⬅︎QiitaのほうはDになっているけどGだよね？)を求める．  
この求めた確率分布からサンプリングを行うことで$x$つまり画像を生成する．  
  
generatorは潜在変数から画像データの確率密度分布を出力するのではない  
generatorは画像データそのものを出力する  
本来確率分布からサンプリングする時には，その分布に従った揺らぎを持って確率論的に値を生成する．(確率をそのまま画像にするから？)  
しかし，generatorの場合は同じ$z$からは常に決定論的に同じ値が出力される．  

この$z$を連続的に空間上で移動させれば，2~3のような画像も連続的な変化がみられる．
 

## 潜在変数の次元 
画像データに対して何次元の潜在変数が必要か？  
データの次元(784)も必要はない  
25個の変化しない潜在変数ベクトルを与えて学習させてみる．  
- zdim = 100: 綺麗な画像が出力される
- zdim = 50: 画像の質は落ちていないが，epochが進むごとに数字が変わってしまう．また，同じ画像が多いような気がする
- zdim = 10: 画像きれい
- zdim = 2: zdim=50でみられた現象がより顕著に現れている
- zdim=1: zdim=2と同じ．だがここまでくると訓練データを模倣してしまっているようにも思える．  
  
ここで起きている問題は**モード崩壊**(mode collapse)と呼ばれる現象．  
generatorの学習に失敗し，訓練データの分布全体を表現できずに，訓練データの最頻値(mode)のみを学習してしまっている．イメージとしては，全国民の期待に応える能力がなく，とりあえず多数派のための政策を作る，みたいなイメージ  
  
zdim=2,1の時は戦時変数の次元が不十分で，訓練データの分布を表現しきれず，特定の画像に集中(1とか)することで，discriminatorを騙すようにしてきた  
  
zdim=50の時には，discriminatorの学習係数のバランスが適切ではなかったためmode collapseが発生してしまっていた．このようにzdimが単純な線形で変化しないのが，パラメータチューニングが難しい一因とも思われる．  
  
99000iteration後に10000サンプル生成させた時の各数字を別のCNNで識別させ，それぞれのzdimでどの数字がいくつ生成されたのかをヒストグラムにしてみると，モード崩壊を起こしていたzdimでは生成される数字が偏ってしまっていることがわかる．  
  
さて，生成データを見ずに学習時の損失関数から学習がうまくいっているかを判断したい．

## まとめ
dcganで高精細な画像を生成することに成功しても，モード崩壊により望む出力を得られない問題がわかった．  
生成結果を確認せずに学習が成功しているかを判断し，モード崩壊をさせない方法が必要． 
  
というわけで，WassersteinGANに次回はトライしてみる．